In [1]:
import jax
import jax.numpy as jnp
import jax.scipy as jsp
from jax import grad, jit, vmap
from jax import random
from jax import jacfwd
import jax_cosmo as jc
import numpy
import scipy
import matplotlib.pyplot as plt
from scipy.stats import rayleigh

In [2]:
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_debug_nans", True)

# Calculate the Fisher Matrix of SN programs

The data are m and measurement uncertainty dm

The model parameters are

Omega: Cosmological
c: calibration
Av, Rv: per supernova dust

The likelihood p(y| Omega, c, Av, Rv) is
N(c,dc) P(Av, Rv) Prod_SN N(m-(M+mu+C(c)+D(Av, Rv), dm_I^2 + dm^2)

The terms are
<partial ^2 lnL / partial alpha partial beta> = -partial X / partial alpha C^{-1} partial X /beta + delta_alpha,beta d2(ln p_alpha)


In [3]:
@jit
def M_sn(wl):
    wl0=5500.
    sig2 = 5e6
    return (wl-wl0)**2/2/sig2/2.5

In [4]:
def pAv():
    return rayleigh
Avscale = 0.05
def lnpAv(x):
    return -2*numpy.log(Avscale) + numpy.log(x) - x*x/2/Avscale**2
def d2lnpAv(x):
    return -1/x/x -1/Avscale/Avscale

In [5]:
@jit
def CCM_a(wl):
    def infrared(x):
        a = 0.574*x**1.61
        return a

    def optical(x):
        y=x-1.82
        a = 1 + 0.17699*y - 0.50447*y**2 - 0.02427*y**3 + 0.72085*y**4+ 0.01979*y**5 - 0.77530*y**6 + 0.32999*y**7
        return a

    def ultraviolet(x):
        a = 1.752 - 0.316*x - 0.104/((x - 4.67)**2 + 0.341)
        return a
        
    x = 1e4/wl
    return jnp.select(condlist=[(x >= 0.3) & (x<=1.1), (x >= 1.1) & (x<=3.3), (x >= 3.3) & (x<=5.9)],
                                choicelist=[infrared(x), optical(x), ultraviolet(x)], default=0.)
def CCM_a_multi(wl):
    return vmap(CCM_a,(0))(wl)

In [6]:
@jit
def CCM_b(wl):
    def infrared(x):
        b = -0.527*x**1.61
        return b

    def optical(x):
        y=x-1.82
        b = 1.41338*y + 2.28305*y**2 + 1.07233*y**3 - 5.38434*y**4 - 0.62251*y**5 + 5.30260*y**6 - 2.09002*y**7
        return b

    def ultraviolet(x):
        b = -3.090 + 1.825*x + 1.206/((x - 4.62)**2 + 0.263)
        return b
        
    x = 1e4/wl
    return jnp.select(condlist=[(x >= 0.3) & (x<=1.1), (x >= 1.1) & (x<=3.3), (x >= 3.3) & (x<=5.9)],
                                choicelist=[infrared(x), optical(x), ultraviolet(x)], default=0.)

def CCM_b_multi(wl):
    return vmap(CCM_b,(0))(wl)

## Chi-sq term (not including dm_int term which is another part of the code)

In [7]:
# For one object one band
def mbar_perfilt(Omega_c, w0, wa, dC, Av, Rv, z, efflam):
    # mu not efficient
    cosmo = jc.Planck15(Omega_c=Omega_c, w0=w0, wa=wa)
    restlam=(efflam+dC[0])/(1+z)
    
    mu = M_sn(restlam) + 10*jnp.log10(1+z) + 5*jnp.log10(jc.background.angular_diameter_distance(cosmo, 1/(1+z)))[0]
    
    # extiction
    A = Av * (CCM_a(restlam)+CCM_b(restlam)/Rv)
    
    # calibration
    return mu + A + dC[1]

def dmbar_perfilt(Omega_c, w0, wa, dC, Av, Rv, z, efflam):
    return grad(mbar_perfilt,(0,1,3,4))(Omega_c, w0, wa, dC, Av, Rv, z, efflam)

def dmbar(Omega_c, w0, wa, dCs, Av, Rv, z, efflams):
    return vmap(dmbar_perfilt, (None, None, None, 0,  None, None, None, 0))(Omega_c, w0, wa, dCs, Av, Rv, z, efflams)
    

def mbar(Omega_c, w0, wa, dCs, Av, Rv, z, efflams):
    return vmap(mbar_perfilt, (None, None, None, 0,  None, None, None, 0))(Omega_c, w0, wa, dCs, Av, Rv, z, efflams)

def mbar_check():
    nsn=10
    efflams = numpy.array([4700., 6420.,  7849.])
    zs = numpy.linspace(0.1,1,nsn)
    Avs = pAV().rvs(size=nsn, scale=AVscale)
    Rvs = numpy.zeros(nsn)+3.1
    dCs= numpy.zeros((3,2))
    Omega_c=0.3
    w0=-1.
    wa=0.
    # ans = mbar_perfilt(Omega_c, w0, wa,dCs[0],Avs[0],Rvs[0],zs[0],efflams[0])
    # dans = dmbar_perfilt(Omega_c, w0, wa ,dCs[0],Avs[0],Rvs[0],zs[0],efflams[0])
    # print(ans)
    # for _ in dans:
    #     print(_)
    ans = mbar(Omega_c, w0, wa,dCs,Avs[0],Rvs[0],zs[0],efflams)
    dans = dmbar(Omega_c, w0, wa ,dCs,Avs[0],Rvs[0],zs[0],efflams)
    # print(ans)
    for _ in dans:
        print(_)

def F_chisq(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data):

    nfilt = len(efflams)
    npars = 1 + 1 + 2*nfilt + len(Avs)
                       
    F=numpy.zeros((npars,npars))

    # For now assume all data have same incertainties
    cov = numpy.zeros((nfilt,nfilt))+dm_int**2
    diag = numpy.identity(nfilt) * dm_data**2
    cov = cov + diag
    # Despite the documentation, the example shows L.T L = cov
    L, lower = jsp.linalg.cho_factor(cov)
    
    # loop over SNe
    snind=0
    for Av , Rv, z in zip( Avs, Rvs, zs):
        dmbars = dmbar(Omega_c, w0, wa, dCs, Av, Rv, z, efflams)
        #Cosmo
        for i in range(2):
            rhs = jsp.linalg.solve_triangular(L, dmbars[i],trans=1,lower=lower)
            F[i,i] += jnp.dot(rhs,rhs)
            #Cosmo
            for j in range(i+1,2):
                lhs = jsp.linalg.solve_triangular(L, dmbars[j],trans=1,lower=lower)
                F[i,j] += jnp.dot(lhs,rhs)
            # Calibrations
            for j in range(nfilt):
                for k in range(2):
                    partial = numpy.zeros(nfilt)
                    partial[j] = dmbars[2][j][k]
                    lhs = jsp.linalg.solve_triangular(L, partial,trans=1,lower=lower)
                    F[i,2+2*j+k] += jnp.dot(lhs,rhs)
            # Av
            lhs = jsp.linalg.solve_triangular(L, dmbars[3],trans=1,lower=lower)
            F[i,2+2*nfilt+snind] += jnp.dot(lhs,rhs)

        #Calibrations
        for i in range(nfilt):
            for l in range(2):
                partial = numpy.zeros(nfilt)
                partial[i] = dmbars[2][i][l]
                rhs = jsp.linalg.solve_triangular(L, partial,trans=1,lower=lower)
                F[2+2*i+l,2+2*i+l] += jnp.dot(rhs,rhs)
                
                # Calibrations
                for j in range(i,nfilt):
                    for k in range(2):
                        partial = numpy.zeros(nfilt)
                        partial[j] = dmbars[2][j][k]
                        lhs = jsp.linalg.solve_triangular(L, partial,trans=1,lower=lower)
                        F[2+2*i+l,2+2*j+k] += jnp.dot(lhs,rhs)
                # Av
                lhs = jsp.linalg.solve_triangular(L, dmbars[3],trans=1,lower=lower)
                F[2+2*i+l,2+2*nfilt+snind] += jnp.dot(lhs,rhs)

        #Av
        rhs = jsp.linalg.solve_triangular(L, dmbars[3],trans=1,lower=lower)
        F[2+2*nfilt+snind,2+2*nfilt+snind] += jnp.dot(rhs,rhs)
            
        snind+=1

    for i in range(npars):
        for j in range(i+1,npars):
            F[j,i]=F[i,j]

    return F
        
def F_chisq_check():
    nsn=10
    efflams = numpy.array([4700., 6420.,  7849.])
    zs = numpy.linspace(0.1,1,nsn)
    Avs = pAV().rvs(size=nsn, scale=AVscale)
    Rvs = numpy.zeros(nsn)+3.1
    dCs= numpy.zeros((3,2))
    Omega_c=0.3
    w0=-1.
    wa=0.
    dm_int=0.1
    dm_data=0.02
    F = F_chisq(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data)
    print(F)

## Chi-sq term for dm_int (Not implemented correctly)

In [ ]:
def lnL_dm_int(dm_int, dm_data, nfilt):
    
    # For now assume all data have same incertainties
    cov = numpy.zeros((nfilt,nfilt))+dm_int**2
    diag = numpy.identity(nfilt) * dm_data**2
    cov = cov + diag
    # Despite the documentation, the example shows L.T L = cov
    L, lower = jsp.linalg.cho_factor(cov)    
    return -0.5/jnp.diagonal(L).prod()**2

def d2ln_dm_int(dm_int, dm_data, nfilt):
    return grad(grad(lnL_dm_int,(0)),(0))(dm_int,dm_data, nfilt)

def F_chisq_dm_int(efflams, dm_int, dm_data, nsn):
    nfilt = len(efflams)

    return -4*nsn*d2ln_dm_int(dm_int, dm_data, nfilt # 4 comes from <>

def F_chisq_dm_int

## A term

In [8]:
def F_Av(Avs):
    ans = []
    for Av in Avs:
        ans.append(-d2lnpAv(Av))
    return ans

def F_Av_test():
    nsn=10
    Avs = pAv().rvs(size=nsn, scale=Avscale)    
    print(Avs, F_Av(Avs))

# F_Av_test()

## Log det term

In [9]:
def lnL_logdet(dm_int, dm_data, efflams):
    nfilt=len(efflams)
    # For now assume all data have same incertainties
    cov = numpy.zeros((nfilt,nfilt))+dm_int**2
    diag = numpy.identity(nfilt) * dm_data**2
    cov = cov + diag
    # Despite the documentation, the example shows L.T L = cov
    L, lower = jsp.linalg.cho_factor(cov)

    # logdet term
    ans = - jnp.log(jnp.diagonal(L)).sum()
    return ans

def F_logdet(dm_int, dm_data, efflams):
    return grad(grad(lnL_logdet,(0)),(0))(dm_int, dm_data, efflams)

def F_logdet_check():
    dm_int=0.1
    dm_data=0.02
    efflams = numpy.array([4700., 6420.,  7849.])
    print(F_logdet(dm_int, dm_data, efflams))

## Combining all contributions to F (for now leaving out dm_int)

In [10]:
def F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data):

    nfilt = len(efflams)
    
    # chisq term fills everything
    F = F_chisq(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data)

    # by hand put in priors for calibration
    for i in range(nfilt):
        F[2+i*nfilt,2+i*nfilt] += 1.
        F[2+i*nfilt+1,2+i*nfilt+1] += 1e4
    
    Favs = F_Av(Avs)
    for i in range(len(Avs)):
        F[2+2*nfilt+i, 2+2*nfilt+i]+=Favs[i]

    return F

In [11]:
cosmo = jc.Planck15()
nsn=100
dm_int =0.1
dm_data =0.02
efflams = numpy.array([4700., 6420.,  7849.])
zs = numpy.linspace(0.1,1,nsn)
Avs = pAv().rvs(size=nsn, scale=Avscale)
Rvs = numpy.zeros(nsn)+3.1
dCs= numpy.zeros((3,2))
Omega_c=cosmo.Omega_c
w0=cosmo.w0
wa=cosmo.wa

F =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data)
print(numpy.linalg.slogdet(F))
Finv= numpy.linalg.inv(F)
print(Finv[0:2,0:2])

I0000 00:00:1708220671.044856       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


(1.0, 815.4223106009849)
[[ 0.00795198 -0.01983934]
 [-0.01983934  0.05059817]]
